[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ruliana/pytorch-katas/blob/main/dan_1/temple_rice_portion_calculator_unrevised.ipynb)

🏮 The Ancient Scroll Unfurls 🏮

THE SACRED RICE PORTION PROPHECY
Dan Level: 1 (Temple Sweeper) | Time: 75 minutes | Sacred Arts: Multi-Feature Tensors, Optimizer Wisdom, Data Scaling

📜 THE MASTER'S CHALLENGE

Young Grasshopper, another sacred duty awaits you in the temple's heart.

The temple's rice stores have been a source of great confusion. Each day, Cook Oh-Pai-Timizer
must calculate the perfect amount of rice to prepare for the monks' meals. Too little,
and the disciples go hungry during their long meditation sessions. Too much, and precious
grains are wasted - a violation of the temple's principles of mindful consumption.

The rice requirements depend on mysterious factors: the number of monks present, the type
of meal being served, and even the season of the year affects appetites!

*CLANG! SPLASH!*

"Oh no! Forgive me!" cries He-Ao-World from the kitchen storage, where rice measurement
scrolls now float in a puddle of spilled water. "I was organizing the portion records
and... well, now some of the measurements have run together! The numbers for winter
feasts are mixed with summer breakfasts, and I can't tell which monks were counted
twice!"

"Rice," muses Master Pai-Torch, appearing silently in the doorway, "is not governed by
a single truth, but by the harmony of many factors. The wise cook learns to weigh
multiple ingredients in the recipe of prediction."

Your sacred duty: Create a model that can predict the perfect rice portions based on
multiple temple variables.

🎯 THE SACRED OBJECTIVES

- [ ] Master multi-feature tensor creation and manipulation
- [ ] Forge a Linear Wisdom layer that handles multiple inputs
- [ ] Learn the ancient art of data scaling and preprocessing
- [ ] Compare different optimization spirits (SGD vs Adam)
- [ ] Practice the sacred ritual of train/test splitting
- [ ] Measure your model's wisdom with evaluation metrics

In [ ]:
# 🍚 THE SACRED IMPORTS

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from typing import Tuple, Dict
import pandas as pd

# Ensure reproducible mystical results
torch.manual_seed(42)
np.random.seed(42)

🍚 THE SACRED RICE DATA GENERATION SCROLL

In [ ]:
def generate_temple_rice_data(n_meals: int = 200, kitchen_chaos: float = 0.1,
                            sacred_seed: int = 42) -> Tuple[torch.Tensor, torch.Tensor, list]:
    """
    Generate observations of temple rice consumption patterns.
    
    Ancient wisdom suggests rice needed follows this sacred formula:
    rice_cups = 0.8 * n_monks + 1.2 * meal_type + 0.3 * season_factor + 2.0
    
    Args:
        n_meals: Number of meal observations to simulate
        kitchen_chaos: Amount of measurement unpredictability
        sacred_seed: Ensures consistent randomness
    
    Returns:
        Tuple of (features, rice_amounts, feature_names)
    """
    torch.manual_seed(sacred_seed)
    np.random.seed(sacred_seed)
    
    # Generate temple meal features
    n_monks = np.random.randint(5, 30, n_meals)  # 5-30 monks per meal
    
    # Meal types: 0=breakfast, 1=lunch, 2=dinner (dinner requires more rice)
    meal_type = np.random.randint(0, 3, n_meals)
    
    # Season factors: 0=spring, 1=summer, 2=autumn, 3=winter (winter=more appetite)
    season = np.random.randint(0, 4, n_meals)
    
    # The sacred rice calculation formula
    base_rice = 2.0
    monk_factor = 0.8
    meal_factor = 1.2
    season_factor = 0.3
    
    rice_needed = (monk_factor * n_monks + 
                  meal_factor * meal_type + 
                  season_factor * season + 
                  base_rice)
    
    # Add kitchen chaos (measurement errors, varying appetites)
    chaos = np.random.normal(0, kitchen_chaos * np.std(rice_needed), n_meals)
    rice_needed = rice_needed + chaos
    
    # Even mystical rice has practical limits
    rice_needed = np.clip(rice_needed, 1.0, 50.0)
    
    # Combine features into tensor
    features = np.column_stack([n_monks, meal_type, season])
    features_tensor = torch.FloatTensor(features)
    rice_tensor = torch.FloatTensor(rice_needed).unsqueeze(1)
    
    feature_names = ['n_monks', 'meal_type', 'season']
    
    return features_tensor, rice_tensor, feature_names

def visualize_rice_wisdom(features: torch.Tensor, rice_amounts: torch.Tensor,
                        feature_names: list, predictions: torch.Tensor = None):
    """Display the sacred patterns of rice consumption."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot each feature vs rice amount
    for i, (ax, feature_name) in enumerate(zip(axes.flat[:3], feature_names)):
        ax.scatter(features[:, i].numpy(), rice_amounts.numpy(), 
                  alpha=0.6, color='brown', s=40)
        ax.set_xlabel(feature_name.replace('_', ' ').title())
        ax.set_ylabel('Rice Cups Needed')
        ax.set_title(f'🍚 Rice vs {feature_name.replace("_", " ").title()}')
        ax.grid(True, alpha=0.3)
    
    # Predictions vs actual in the 4th subplot
    if predictions is not None:
        ax = axes.flat[3]
        ax.scatter(rice_amounts.numpy(), predictions.detach().numpy(), 
                  alpha=0.6, color='gold', s=40)
        # Perfect prediction line
        min_val = min(rice_amounts.min(), predictions.min())
        max_val = max(rice_amounts.max(), predictions.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8)
        ax.set_xlabel('Actual Rice Cups')
        ax.set_ylabel('Predicted Rice Cups')
        ax.set_title('🎯 Predictions vs Reality')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Generate the sacred data
features, rice_amounts, feature_names = generate_temple_rice_data()
print(f"Generated {len(features)} sacred meal observations")
print(f"Features: {feature_names}")
print(f"Feature shapes: {features.shape}")
print(f"Rice amounts shape: {rice_amounts.shape}")
print(f"Rice range: {rice_amounts.min():.2f} to {rice_amounts.max():.2f} cups")

# Visualize the sacred patterns
visualize_rice_wisdom(features, rice_amounts, feature_names)

🧮 THE SACRED ART OF DATA PREPARATION

In [ ]:
def prepare_sacred_data(features: torch.Tensor, rice_amounts: torch.Tensor, 
                      test_size: float = 0.2) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Prepare the data for training by scaling and splitting.
    
    Args:
        features: Input features tensor
        rice_amounts: Target rice amounts tensor
        test_size: Fraction of data to use for testing
    
    Returns:
        Tuple of (X_train, X_test, y_train, y_test)
    """
    # TODO: Convert tensors to numpy for sklearn compatibility
    X_np = None
    y_np = None
    
    # TODO: Split the data into training and testing sets
    # Hint: Use train_test_split with random_state=42 for reproducibility
    X_train_np, X_test_np, y_train_np, y_test_np = None, None, None, None
    
    # TODO: Scale the features using StandardScaler
    # Hint: Fit the scaler on training data only, then transform both sets
    scaler = None
    X_train_scaled = None
    X_test_scaled = None
    
    # Convert back to tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    X_test_tensor = torch.FloatTensor(X_test_scaled)
    y_train_tensor = torch.FloatTensor(y_train_np).unsqueeze(1)
    y_test_tensor = torch.FloatTensor(y_test_np).unsqueeze(1)
    
    print(f"Training set size: {len(X_train_tensor)}")
    print(f"Testing set size: {len(X_test_tensor)}")
    
    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor

# Prepare the sacred data
# X_train, X_test, y_train, y_test = prepare_sacred_data(features, rice_amounts)
print("📊 Data preparation wisdom awaits your implementation...")

🏮 THE RICE PORTION PREDICTOR

In [ ]:
class RicePortionPredictor(nn.Module):
    """A mystical artifact for understanding sacred rice consumption patterns."""
    
    def __init__(self, input_features: int = 3):
        super(RicePortionPredictor, self).__init__()
        # TODO: Create the Linear Wisdom layer for multi-feature input
        # Hint: input_features should be 3 (n_monks, meal_type, season)
        # Output should be 1 (rice cups needed)
        self.linear_wisdom = None
    
    def divine_rice_portions(self, temple_features: torch.Tensor) -> torch.Tensor:
        """Channel your understanding through the mystical network."""
        # TODO: Pass the multi-feature input through your Linear Wisdom
        # Remember: the network must handle multiple inputs simultaneously
        return None
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """The sacred forward pass - this is called automatically by PyTorch."""
        return self.divine_rice_portions(x)

# Create your mystical predictor
rice_predictor = RicePortionPredictor()
print("🍚 Your Rice Portion Predictor has been forged!")
print(f"Sacred parameters: {sum(p.numel() for p in rice_predictor.parameters())}")
print(f"Model architecture: {rice_predictor}")

⚔️ THE DUAL OPTIMIZER TRAINING RITUAL

In [ ]:
def train_rice_predictor(model: nn.Module, X_train: torch.Tensor, y_train: torch.Tensor,
                       X_test: torch.Tensor, y_test: torch.Tensor,
                       optimizer_name: str = 'sgd', epochs: int = 1000, 
                       learning_rate: float = 0.01) -> Dict[str, list]:
    """
    Train the rice portion prediction model.
    
    Args:
        optimizer_name: 'sgd' or 'adam' - choose your optimization spirit
    
    Returns:
        Dictionary containing training history
    """
    # TODO: Choose your loss calculation method
    # Hint: Mean Squared Error works well for regression
    criterion = None
    
    # TODO: Choose your optimization spirit
    # Hint: Compare SGD vs Adam - they have different strengths
    if optimizer_name.lower() == 'sgd':
        optimizer = None
    elif optimizer_name.lower() == 'adam':
        optimizer = None
    else:
        raise ValueError("Choose 'sgd' or 'adam' as your optimization spirit")
    
    history = {'train_loss': [], 'test_loss': []}
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        
        # TODO: CRITICAL - Clear the gradient spirits from previous cycle
        
        # TODO: Forward pass - get predictions
        train_predictions = None
        
        # TODO: Compute the training loss
        train_loss = None
        
        # TODO: Backward pass - compute gradients
        
        # TODO: Update parameters
        
        # Evaluation phase (no gradient computation needed)
        model.eval()
        with torch.no_grad():
            test_predictions = model(X_test)
            test_loss = criterion(test_predictions, y_test)
        
        history['train_loss'].append(train_loss.item())
        history['test_loss'].append(test_loss.item())
        
        # Report progress to the masters
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}] | '
                  f'Train Loss: {train_loss.item():.4f} | '
                  f'Test Loss: {test_loss.item():.4f}')
            
            if train_loss.item() < 5.0:
                print(f"🍚 Cook Oh-Pai-Timizer approves - the {optimizer_name.upper()} spirits are working!")
    
    return history

# Prepare for the dual training ritual
print("🧘 The Dual Optimizer Training Ritual awaits...")
print("Master Pai-Torch whispers: 'Each optimization spirit has its own wisdom to offer.'")

# Training begins here - uncomment after implementing the TODOs
# sgd_history = train_rice_predictor(rice_predictor, X_train, y_train, X_test, y_test, 
#                                   optimizer_name='sgd', learning_rate=0.01)

# Create a fresh model for Adam training
# rice_predictor_adam = RicePortionPredictor()
# adam_history = train_rice_predictor(rice_predictor_adam, X_train, y_train, X_test, y_test,
#                                    optimizer_name='adam', learning_rate=0.01)

📊 VISUALIZATION OF THE SACRED TRAINING COMPARISON

In [ ]:
def compare_optimization_spirits(sgd_history: Dict, adam_history: Dict):
    """Compare the wisdom of different optimization spirits."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Training loss comparison
    axes[0, 0].plot(sgd_history['train_loss'], label='SGD', color='blue', linewidth=2)
    axes[0, 0].plot(adam_history['train_loss'], label='Adam', color='red', linewidth=2)
    axes[0, 0].set_title('🏃 Training Loss: The Race of Optimizers')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Training Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Test loss comparison
    axes[0, 1].plot(sgd_history['test_loss'], label='SGD', color='blue', linewidth=2)
    axes[0, 1].plot(adam_history['test_loss'], label='Adam', color='red', linewidth=2)
    axes[0, 1].set_title('🎯 Test Loss: The True Wisdom Test')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Test Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Final comparison summary
    optimizers = ['SGD', 'Adam']
    final_train_losses = [sgd_history['train_loss'][-1], adam_history['train_loss'][-1]]
    final_test_losses = [sgd_history['test_loss'][-1], adam_history['test_loss'][-1]]
    
    x_pos = np.arange(len(optimizers))
    width = 0.35
    
    axes[1, 0].bar(x_pos - width/2, final_train_losses, width, label='Train Loss', color='lightblue')
    axes[1, 0].bar(x_pos + width/2, final_test_losses, width, label='Test Loss', color='lightcoral')
    axes[1, 0].set_title('⚡ Final Battle: Which Spirit Wins?')
    axes[1, 0].set_xlabel('Optimization Spirit')
    axes[1, 0].set_ylabel('Final Loss')
    axes[1, 0].set_xticks(x_pos)
    axes[1, 0].set_xticklabels(optimizers)
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Remove empty subplot
    axes[1, 1].remove()
    
    plt.tight_layout()
    plt.show()

# Uncomment after training is complete
# compare_optimization_spirits(sgd_history, adam_history)

⚡ THE TRIALS OF MASTERY

## Trial 1: Multi-Feature Mastery
- [ ] Model accepts 3-feature input and produces 1-feature output
- [ ] Training loss decreases consistently for both optimizers
- [ ] Test loss remains close to training loss (no severe overfitting)
- [ ] Final test loss below 5.0 for at least one optimizer

## Trial 2: Optimizer Wisdom Comparison
- [ ] Both SGD and Adam successfully train the model
- [ ] You can explain which optimizer performed better and why
- [ ] Model parameters reflect reasonable rice portion relationships

## Trial 3: Understanding Test

In [ ]:
def test_your_rice_wisdom(model, X_test, y_test):
    """Master Pai-Torch's comprehensive evaluation of your understanding."""
    model.eval()
    
    # Test shape compatibility
    test_features = torch.FloatTensor([[15, 2, 1]])  # 15 monks, dinner, summer
    prediction = model(test_features)
    assert prediction.shape == (1, 1), "The rice prediction shapes must align!"
    
    # Test on full test set
    with torch.no_grad():
        test_predictions = model(X_test)
        test_mse = mean_squared_error(y_test.numpy(), test_predictions.numpy())
        test_r2 = r2_score(y_test.numpy(), test_predictions.numpy())
    
    print(f"Test MSE: {test_mse:.4f}")
    print(f"Test R²: {test_r2:.4f}")
    
    # Check if model learned reasonable relationships
    params = list(model.parameters())
    weights = params[0].data.numpy().flatten()
    bias = params[1].data.numpy().item()
    
    print(f"\nLearned weights: {weights}")
    print(f"Learned bias: {bias:.3f}")
    
    # The first weight should be positive (more monks = more rice)
    assert weights[0] > 0, "More monks should need more rice!"
    
    # Test should achieve reasonable accuracy
    assert test_r2 > 0.8, f"R² of {test_r2:.3f} suggests the model needs more training!"
    
    print("\n🎉 Cook Oh-Pai-Timizer beams with pride - your rice wisdom is profound!")
    
    return test_mse, test_r2

# Test your wisdom after training
# test_mse, test_r2 = test_your_rice_wisdom(rice_predictor, X_test, y_test)

🌸 THE FOUR PATHS OF MASTERY: PROGRESSIVE EXTENSIONS

## Extension 1: Master Ao-Tougrad's Feature Engineering
*"The wise predictor creates new features from existing wisdom."*

*A shadow falls across your work as Master Ao-Tougrad appears*

"Young grasshopper, your multi-feature mastery grows strong. But consider this:
what happens when monks arrive together in groups? When winter dinners combine
the appetite of cold weather with the heartiness of evening meals? Sometimes
the true wisdom lies not in more features, but in the interaction between them."

**NEW CONCEPTS**: Feature engineering, interaction terms, polynomial features  
**DIFFICULTY**: +15% (still Dan 1, but smarter features)

In [ ]:
def create_interaction_features(features: torch.Tensor) -> torch.Tensor:
    """
    Create interaction features to capture complex relationships.
    
    Args:
        features: Original features [n_monks, meal_type, season]
    
    Returns:
        Enhanced features with interactions
    """
    # TODO: Add interaction terms
    # Hint: monks * meal_type captures "dinner needs more rice per monk"
    # Hint: monks * season captures "winter monks eat more"
    # Hint: meal_type * season captures "winter dinners are largest"
    pass

# TRIAL: Train your model with engineered features
# SUCCESS: Achieve better R² score with the same model architecture

## Extension 2: Cook Oh-Pai-Timizer's Batch Cooking Wisdom
*"A master chef prepares many meals simultaneously, not one at a time!"*

*Cook Oh-Pai-Timizer appears with a large pot and ladle*

"Grasshopper, your single-meal predictions serve well for daily planning. But what
happens during festival weeks when we must prepare dozens of meals in advance?
The temple's efficiency comes from batch processing - predicting rice needs for
entire weeks at once!"

**NEW CONCEPTS**: Batch prediction, vectorized operations, efficiency optimization  
**DIFFICULTY**: +25% (still Dan 1, but thinking in batches)

In [ ]:
def generate_festival_week_data(n_weeks: int = 5) -> torch.Tensor:
    """
    Generate meal data for entire festival weeks.
    Each week has 21 meals (7 days × 3 meals/day).
    
    Returns:
        Batch of meal features for multiple weeks
    """
    # TODO: Generate batch data for festival weeks
    # Hint: Your existing model should handle this automatically
    pass

def predict_festival_rice_needs(model, festival_data: torch.Tensor) -> Dict:
    """
    Predict rice needs for entire festival weeks.
    
    Returns:
        Dictionary with weekly totals and daily breakdowns
    """
    # TODO: Make batch predictions and organize by week
    # Hint: Reshape predictions to separate weeks and days
    pass

# TRIAL: Predict rice needs for multiple festival weeks
# SUCCESS: Efficiently process batches of 100+ meals simultaneously

## Extension 3: He-Ao-World's Noisy Data Challenge
*"These old records seem to have some... inconsistencies in them."*

*He-Ao-World shuffles over with a stack of water-damaged scrolls*

"Oh dear! I'm afraid some of our historical rice records got mixed up in the
monsoon season. Some measurements might be in different units, others might
have extra digits, and I think a few guest counts include the temple dogs by
mistake! Can your model still learn from imperfect data?"

**NEW CONCEPTS**: Robust training, outlier detection, data cleaning  
**DIFFICULTY**: +35% (still Dan 1, but messy real-world data)

In [ ]:
def add_realistic_noise(features: torch.Tensor, rice_amounts: torch.Tensor, 
                       noise_level: float = 0.1) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Add realistic data inconsistencies to simulate He-Ao-World's "accidents".
    
    Args:
        noise_level: Fraction of data to corrupt
    
    Returns:
        Noisy features and rice amounts
    """
    # TODO: Add various types of realistic noise:
    # - Some monk counts doubled (counted twice)
    # - Some rice amounts in wrong units
    # - A few completely wrong outliers
    pass

def robust_training_with_outliers(model, X_train, y_train, X_test, y_test):
    """
    Train model to be robust against outliers and noise.
    
    Returns:
        Training history with robustness metrics
    """
    # TODO: Implement training strategies that handle noisy data
    # Hint: Consider using different loss functions or data cleaning
    pass

# TRIAL: Train on noisy data and maintain good performance
# SUCCESS: Model achieves similar accuracy despite data corruption

## Extension 4: The Temple's Rice Inventory Optimization
*"Prediction is wisdom, but optimization is mastery."*

*Master Pai-Torch gestures toward the temple's rice storage*

"Young grasshopper, your prediction wisdom has grown deep. But the temple asks
a greater question: 'Given our rice storage limits, how should we schedule our
meals to minimize waste while ensuring no monk goes hungry?' This is the path
from prediction to optimization."

**NEW CONCEPTS**: Constraint optimization, resource allocation, decision making  
**DIFFICULTY**: +45% (still Dan 1, but strategic thinking)

In [ ]:
def optimize_meal_schedule(model, available_rice: float, 
                         meal_options: torch.Tensor, 
                         max_meals: int = 10) -> Dict:
    """
    Optimize meal scheduling given rice constraints.
    
    Args:
        available_rice: Total rice cups available
        meal_options: Tensor of possible meal configurations
        max_meals: Maximum number of meals to schedule
    
    Returns:
        Optimal meal schedule with rice allocation
    """
    # TODO: Use your model to predict rice needs for all meal options
    # TODO: Select optimal combination that fits within rice budget
    # Hint: This is a combinatorial optimization problem
    pass

def visualize_rice_optimization(optimal_schedule: Dict, available_rice: float):
    """
    Visualize the optimal meal schedule and rice allocation.
    """
    # TODO: Create visualization showing:
    # - Selected meals and their rice requirements
    # - Rice budget utilization
    # - Comparison with other possible schedules
    pass

# TRIAL: Optimize meal scheduling for different rice budgets
# SUCCESS: Maximize meals served while staying within rice limits
# MASTERY: Understand how prediction models enable optimization decisions

🔥 CORRECTING YOUR FORM: A SCALING IMBALANCE

Cook Oh-Pai-Timizer observes your data preparation with concern. "Young grasshopper, your ingredient measurements seem... unbalanced. See how some features dominate others like salt overwhelming delicate spices?"

A previous disciple left this flawed data preparation ritual. The model struggles to learn - can you restore proper technique?

In [ ]:
def unbalanced_data_preparation(features, rice_amounts):
    """This data preparation has lost its balance - your form needs correction! 🥋"""
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        features.numpy(), rice_amounts.numpy(), test_size=0.2, random_state=42
    )
    
    # Convert back to tensors - but something is missing...
    X_train_tensor = torch.FloatTensor(X_train)
    X_test_tensor = torch.FloatTensor(X_test)
    y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
    y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1)
    
    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor

# Cook Oh-Pai-Timizer's guidance: "What sacred step is missing from this preparation?"
# Hint: The features have very different scales - some are much larger than others...
# Hint: n_monks (5-30) vs meal_type (0-2) vs season (0-3)
# Can you identify why the model struggles and fix the missing step?

🎊 COMPLETION CEREMONY

When you have mastered all trials, you will have achieved:

- **Multi-Feature Mastery**: Working with multi-dimensional input tensors
- **Data Preparation Wisdom**: Scaling, splitting, and preprocessing data
- **Optimizer Comparison**: Understanding SGD vs Adam optimization spirits
- **Evaluation Metrics**: Using MSE, R², and other assessment tools
- **Batch Processing**: Handling multiple predictions simultaneously
- **Real-World Challenges**: Dealing with noisy, imperfect data

Cook Oh-Pai-Timizer nods with satisfaction: *"The grasshopper has learned that even the humblest grain of rice follows the deeper patterns of the universe. Multi-feature wisdom and optimization spirits - these are the foundations upon which greater temples are built."*

🍚 **Your Dan 1 Temple Sweeper mastery deepens!** 🍚

The path to Dan 2 grows clearer with each lesson learned...